# PREDICTION WEB APPLICATION

### Importing liabriess 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import os
import glob
import json 
import requests
from bs4 import BeautifulSoup
%matplotlib inline

#### Creating function for Downloading Data For Training and Testing The Model

In [2]:
# os.listdir('./Prediction_train&test_Data')
def load_data(url):
    
    folderName = './Prediction_train&test_Data/'
    fileName = 'league_data.json'
    path = os.path.join(folderName, fileName)

    if os.path.exists(path):
        files = glob.glob('./Prediction_train&test_Data/*')
        for f in files:
            os.remove(f)

    response = requests.get(url)
    # print(path)

    with open(path, 'wb') as outfile:
        outfile.write(response.content)
     

        
url = 'https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14001'
load_data(url)

#### Reading Json Files 

In [3]:
folderName = './Prediction_train&test_Data/'
fileName = 'league_data.json'
path = os.path.join(folderName, fileName)

with open(path, 'r') as outfile:
    json_obj = json.load(outfile)

In [4]:
clubes = json_obj['teams']
matches = json_obj['matches']
teams = []
resulte = []
wonmarke = []

count = 0
for key, val in matches.items():
    
    teams.append({key: []})
    resulte.append({key: []})
    wonmarke.append({key: []})
    
    for x in val:
        
        teams[count][key].append([val[x]['teamA'], val[x]['teamB']])
        resulte[count][key].append(val[x]['result'])
        wonmarke[count][key].append(val[x]['wonmarkets'])
        
    count +=1

In [5]:
# json_obj
# teams
# resulte
# wonmarke

#### Wranggling Data into Data Base

- Data
    - columns:
        - teams
        - winners(0 home, 1 draw, 2 away)



In [6]:
# resulte

In [7]:
# wonmarke
# teams

In [8]:
# events = []
events = {}
teams_index = {}
for x in range(len(teams)):
    for y, obj in teams[x].items():
        events[y] = []
        teams_index[y] = []
        for i in obj:
            teamA = clubes[i[0]]['fifa']
            teamB = clubes[i[1]]['fifa']
            events[y].append(f'{teamA}-{teamB}')
            teams_index[y].append('-'.join(i))

In [9]:
score = {}
for x in range(len(wonmarke)):
    for y, string in wonmarke[x].items():
        score[y] = []
        for j in string:
            resulte_win = j.split(',')[0]
            score[y].append(resulte_win)

In [10]:
# score
# events
socore_dataframe = pd.DataFrame(score)
events_dataframe = pd.DataFrame(events)
teams_index_dataframe = pd.DataFrame(teams_index)

In [11]:
socore_dataframe.head()

,1,2,3
0,Away,Home,Away
1,Away,Home,Home
2,Home,Home,Away
3,Draw,Home,Draw
4,Away,Away,Home


In [12]:
events_dataframe.head()

,1,2,3
0,EVE-TOT,LIV-LEI,NWC-BUR
1,ARS-ASV,ASV-NWC,SOU-NOR
2,LEI-BUR,BRI-MNU,ARS-LIV
3,BRN-NOR,CHE-CRY,WOL-LEE
4,NWC-CHE,TOT-WOL,MNU-CHE


In [13]:
teams_index_dataframe.head()

,1,2,3
0,49-51,45-63,53-64
1,48-60,60-53,52-66
2,63-64,71-50,48-45
3,972-66,47-55,525-827
4,53-47,51-525,50-47


In [14]:
def melt_table(dataframe):
    dataframe['index'] = 1
    values_vars = [str(x) for x in range(1, dataframe.shape[1])]
    resulte = pd.melt(dataframe, id_vars=['index'], value_vars=values_vars)
    return resulte

In [15]:
new_events_df = melt_table(events_dataframe).rename(columns={'value': 'events', 'variable': 'weeks'})[['events','weeks']]
new_score_df = melt_table(socore_dataframe).rename(columns={'value': 'score', 'variable': 'weeks'})[['score','weeks']]
teams_index_df = melt_table(teams_index_dataframe).rename(columns={'value': 'teams_index', 'variable': 'weeks'})[['teams_index','weeks']]

In [16]:
new_events_df.head()

,events,weeks
0,EVE-TOT,1
1,ARS-ASV,1
2,LEI-BUR,1
3,BRN-NOR,1
4,NWC-CHE,1


In [17]:
new_score_df.head()

,score,weeks
0,Away,1
1,Away,1
2,Home,1
3,Draw,1
4,Away,1


In [18]:
teams_index_df.head()

,teams_index,weeks
0,49-51,1
1,48-60,1
2,63-64,1
3,972-66,1
4,53-47,1


In [19]:
events_score_df = pd.concat([new_events_df, new_score_df, teams_index_df], axis=1, join="inner").iloc[:, np.r_[0:2, 2, 4]] # [['events', 'score', 'teams_index']]

In [20]:
events_score_df.head(6)

,events,weeks,score,teams_index
0,EVE-TOT,1,Away,49-51
1,ARS-ASV,1,Away,48-60
2,LEI-BUR,1,Home,63-64
3,BRN-NOR,1,Draw,972-66
4,NWC-CHE,1,Away,53-47
5,CRY-MNU,1,Away,55-50


#### Saving Data Into CSV file

In [21]:
events_score_df.to_csv('./Prediction_train&test_Data/premier_leage_data.csv', index=False)